# NLP 2 LAB 03 -  Semantic Search

In [1]:
%%capture
!pip install beir tensorflow_text sentence_transformers hnswlib

In [2]:
from beir import util as b_util, LoggingHandler
from beir.retrieval import models
from beir.datasets.data_loader import GenericDataLoader
from beir.retrieval.evaluation import EvaluateRetrieval
from beir.retrieval.search.dense import DenseRetrievalExactSearch as DRES
from sentence_transformers import SentenceTransformer
from sentence_transformers import util
from tqdm import tqdm
from time import time

import numpy as np
import logging
import random

random.seed(42)

/usr/local/lib/python3.7/dist-packages/beir/util.py:2: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


## (8 points) Create a searchable index
---

In [3]:

#### Download dbpedia-entity.zip dataset and unzip the dataset
dataset = "dbpedia-entity"
url = "https://public.ukp.informatik.tu-darmstadt.de/thakur/BEIR/datasets/{}.zip".format(dataset)
out_dir = "."
data_path = b_util.download_and_unzip(url, out_dir)

#### Provide the data_path where dbpedia-entity has been downloaded and unzipped
corpus, queries, qrels = GenericDataLoader(data_folder=data_path).load(split="test")

  0%|          | 0/4635922 [00:00<?, ?it/s]

### (1 point) Explain the data structure. What are the three values returned by Beir, and how are they presented.

In [4]:
# Display dataset features
print("Number of documents in the corpus: {}".format(len(corpus)))
print("Number of queries: {}".format(len(queries)))
print("Number of qrels: {}".format(len(qrels)))

# Display a sample query
print("Sample query: {}".format(next(iter(queries.items()))))

# Display a sample document
print("Sample document: {}".format(next(iter(corpus.items()))))

# Display a sample qrel
print("Sample qrel: {}".format(next(iter(qrels.items()))))

Number of documents in the corpus: 4635922
Number of queries: 400
Number of qrels: 400
Sample query: ('INEX_LD-2009022', 'Szechwan dish food cuisine')
Sample document: ('<dbpedia:Animalia_(book)>', {'text': "Animalia is an illustrated children's book by Graeme Base. It was originally published in 1986, followed by a tenth anniversary edition in 1996, and a 25th anniversary edition in 2012. Over three million copies have been sold.   A special numbered and signed anniversary edition was also published in 1996, with an embossed gold jacket.", 'title': 'Animalia (book)'})
Sample qrel: ('INEX_LD-2009022', {'<dbpedia:Afghan_cuisine>': 0, '<dbpedia:Akan_cuisine>': 0, '<dbpedia:Ambuyat>': 0, '<dbpedia:American_Chinese_cuisine>': 1, '<dbpedia:Ants_climbing_a_tree>': 2, '<dbpedia:Baingan_bharta>': 1, '<dbpedia:Bamischijf>': 0, '<dbpedia:Black_cardamom>': 0, '<dbpedia:Brazilian_cuisine>': 0, '<dbpedia:British_cuisine>': 0, '<dbpedia:Caribbean_cuisine>': 0, '<dbpedia:Cellophane_noodles>': 1, '<db

**Le dataset est constitué de trois composantes :**
- Dictionnaire de *4 635 922* documents issus des pages de DBpedia avec le contenu `text` et le titre `title` (corpus)
- Dictionnaire de *400* requêtes (queries)
- Les pages pertinentes vers lesquelles chaque requête est censée rediriger (qrels)

### (2 points) To ease the problem, extract all the document from the corpus which are relevant to at least one query. Then, add 100K random documents which are not relevant to any query. Make sure the process is reproducible by setting the random seed on whatever random sampling method you use.

In [5]:
relevant_docs = set()
for qid, q_rels in qrels.items():
    for docid, num in q_rels.items():
        if num > 0:
            relevant_docs.add(docid)

# Add 100K random documents which are not relevant to any query
random_docs = random.sample(list(set(corpus.keys()) - relevant_docs), 100_000)

# Create a new corpus with the relevant and random documents
new_corpus = {docid: corpus[docid] for docid in relevant_docs.union(random_docs)}

In [6]:
# Display a sample of the new corpus
next(iter(new_corpus.items()))

('<dbpedia:Parties_in_the_European_Council_during_2007>',
 {'text': 'This article describes the party affiliations of leaders of each member-state represented in the European Council during the year 2007. The list below gives the political party that each head of government, or head of state, belongs to at the national level, as well as the European political alliance to which that national party belongs. The states are listed from most to least populous.',
  'title': 'Parties in the European Council during 2007'})

Now, we should be ready to start experimenting with our smaller dataset. Use the sentence-transformers library to index your dataset. As queries and documents are different, use an asymetric similarity models. Pick one model across the ones proposed. Make sure to document your choice, and why you picked it (because of accuracy, speed, ...).

In [7]:
%%capture
model = SentenceTransformer('msmarco-distilbert-base-v4').to("cuda")

Nous avons choisi ce model car il s'agit de celui avec la meilleure accuracy parmi les modèles entraînés pour la *cosine similarity*. On perd en vitesse d'exécution par requête mais compte tenu de la puissance de calcul de Google Colab, du GPU et des optimizations appliquées ci-dessous, cette perte est négligeable. 

### (2 points) Embed the reduced corpus and the queries using the chosen model.

In [8]:
corpus_embeddings = model.encode([doc["text"] for doc in new_corpus.values()], convert_to_tensor=True)
query_embeddings = model.encode(list(queries.values()), convert_to_tensor=True)

print("We eventually get embeddings of dimension:", corpus_embeddings.shape)

We eventually get embeddings of dimension: torch.Size([114877, 768])


In [9]:
# Optimizing the queries:
# We send the tensors to the cuda device and normalize embeddings
# so we can compute the inner product instead of the cosine

corpus_embeddings = corpus_embeddings.to('cuda')
corpus_embeddings = util.normalize_embeddings(corpus_embeddings)

query_embeddings = query_embeddings.to('cuda')
query_embeddings = util.normalize_embeddings(query_embeddings)

In [10]:
# Update qrels with new corpus ids and query ids
new_qrels = {}
for id, (qid, q_rels) in enumerate(qrels.items()):
    new_qrels[id] = {}
    for docid, val in q_rels.items():
        if docid in new_corpus and val > 0:
            new_qrels[id][list(new_corpus.keys()).index(docid)] = val

# Display a sample of the new qrels
next(iter(new_qrels.items()))

(0,
 {49745: 1,
  32070: 2,
  73068: 1,
  108771: 1,
  93770: 1,
  16324: 1,
  4871: 2,
  32725: 1,
  64315: 2,
  37610: 1,
  102952: 1,
  56016: 2,
  12020: 1,
  38366: 2,
  62557: 1,
  19081: 1,
  38431: 2,
  46869: 1,
  52860: 1,
  107593: 1,
  52202: 1,
  4643: 2,
  105760: 2,
  113200: 1,
  36073: 2,
  431: 1,
  65520: 1,
  108446: 1,
  30843: 1,
  72985: 2,
  47488: 2,
  91427: 1,
  36707: 1,
  59909: 1,
  47053: 1})

### (3 points) Using the annotated set of queries, compute the Mean Average Precision (MAP) @100 as well as the average time per query.

In [11]:
start = time()
results = util.semantic_search(query_embeddings, corpus_embeddings, top_k=100, score_function=util.dot_score)
end = time()

avg_time = (end - start) / len(query_embeddings)

print("The average time for {} queries is {:.4f} seconds".format(len(query_embeddings), avg_time) )

The average time for 400 queries is 0.0002 seconds


In [12]:
# Let us see what results look like
results[0][:10]

[{'corpus_id': 65520, 'score': 0.613796055316925},
 {'corpus_id': 36073, 'score': 0.4821910560131073},
 {'corpus_id': 12020, 'score': 0.4738737940788269},
 {'corpus_id': 21630, 'score': 0.44950756430625916},
 {'corpus_id': 102342, 'score': 0.44849568605422974},
 {'corpus_id': 49745, 'score': 0.4403640925884247},
 {'corpus_id': 100599, 'score': 0.43535542488098145},
 {'corpus_id': 52860, 'score': 0.42014870047569275},
 {'corpus_id': 34606, 'score': 0.41908279061317444},
 {'corpus_id': 64315, 'score': 0.41114839911460876}]

In [13]:
def map_at_100(hits, qrels):
    """
    Take the top hits for each query and returns the
    Mean Average Precision for the top 100 ranks

    Args:
      - hits (list[list[dict[str, float]]]): The results of the query
      - qrels (dict[int, list[dict[str, int]]]): The actual relevant documents
    Returns:
      The computed MAP@100 (float)

    """
    MAP = 0
    for query_id, _ in enumerate(hits):
        avg_precision, relevants_docs, K = 0, 0, 0
    
        for doc in hits[query_id]:
            K += 1

            is_relevant = 1 if doc["corpus_id"] in qrels[query_id] else 0
            relevants_docs += is_relevant
            precision = relevants_docs / K
            avg_precision += precision * is_relevant

        if relevants_docs > 0:
            avg_precision /= relevants_docs

        MAP += avg_precision
    
    return MAP / len(hits)

MAP100 = map_at_100(results, new_qrels)

print("The MAP@100 of our model is %.4f" % MAP100)

The MAP@100 of our model is 0.6307


## (4 points) Approximate nearest neighbours
---

### (3 points) Find a good set of parameters for the chosen ANN library and compute the MAP@100.


In [14]:
import hnswlib

In [15]:
# HNSW doesn't support cuda tensors
query_embeddings = query_embeddings.cpu()
corpus_embeddings = corpus_embeddings.cpu()

# 'ip' for Inner Product because we previously normalized embeddings
index = hnswlib.Index(space = 'ip', dim = corpus_embeddings.shape[1])

index.init_index(max_elements=corpus_embeddings.shape[0], ef_construction=400, M=64)

index.add_items(corpus_embeddings, list(range(corpus_embeddings.shape[0])))

In [16]:
corpus_ids, scores = index.knn_query(query_embeddings, k=100)

In [17]:
# We convert the results into a usable format
ann_results = [[{"corpus_id":id} for id in doc] for doc in corpus_ids]

MAP100 = map_at_100(ann_results, new_qrels)
print("The MAP@100 for the ANN model is : %.4f" % MAP100)

The MAP@100 for the ANN model is : 0.6275


### (1 point) explain what the parameters you picked are, and why you chose them.

Les paramètres de notre modèle KNN sont :
- `M` le nombre de liens bidirectionnels créés pour chaque nouvel élément pendant la construction. Ce paramètre ce situe généralement entre 2 et 100 et doit être proportionnel à la dimension des vecteurs.
- `ef_construction` determine le rapport vitesse/accuracy lors de la construction du modèle, un plus grand `ef` impliquant une meilleure accuracy mais une recherche plus lente, et vice-versa.

Nos vecteurs `corpus_embeddings` et `query_embeddings` ayant une taille élevée, il est conseillé pour `M` de prendre des valeurs entre *48* et *64*. Quant à `ef_construction`, il doit être supérieur au nombre de voisins qu'on souhaite retourner (100) et ne doit pas dépasser la taille du dataset. Nous prendrons donc une valeur suffisamment élevée dans cet intervalle pour conserver une bonne accuracy, la recherche étant déjà suffisamment rapide.

En outre, nous avons repris les paramètres conseillés par les exemples de la documentation de Sentence-Transformers.

### (Bonus) Play with these parameters and plot a speed vs MAP curve.